## Generating Dataset

In [439]:
import random
from collections import Counter, defaultdict
import numpy as np


def generate_correct_sample(max_length=20):
    n = random.randint(1, np.ceil(max_length / 3))
    return ('a' * n) + ('b' * n) + ('c' * n)

def create_dataset(num_samples, max_length=20):
    dataset = []
    
    # Generate samples of the form a^n b^n c^n
    while len(dataset) < num_samples:
        sample = generate_correct_sample(max_length)
        if len(sample) < max_length + 1:
            dataset.append((sample, 1))
        
    # Generate counterexamples with slight variations
    count_variations = [0,0,0]
    while len(dataset) < 2 * num_samples:
        probabilities = [0.7, 0.15, 0.15]
        variation_type = np.random.choice([0, 1, 2], p=probabilities)
        # Count the number of variations of each type
        if variation_type == 0:
            # Generate counterexample by n randomly selected characters
            n = random.randint(1, np.ceil(max_length / 3))
            chars = random.choices(['a', 'b', 'c'], k=n)
            counter_example = ''.join(chars)
        elif variation_type == 1:
            # Remove one character from the sample
            sample = generate_correct_sample(max_length)
            idx = random.randint(0, len(sample) - 1)
            counter_example = sample[:idx] + sample[idx+1:]
        else:
            # Add one character to the sample
            sample = generate_correct_sample(max_length)
            idx = random.randint(0, len(sample) - 1)
            counter_example = sample[:idx] + random.choice(['a', 'b', 'c']) + sample[idx:]
            
        # Check whether counter example isnt randomly a correct sample
        counter = Counter(counter_example)
        if counter['a'] == counter['b'] == counter['c']:
            continue
        
        # Check if the counterexample is not too long
        if len(counter_example) < 21:
            count_variations[variation_type] += 1
            dataset.append((counter_example, 0))  # Append the counterexample along with its label 0
       
    # Shuffle the dataset 
    random.shuffle(dataset)
    print("Dataset created with {} samples.".format(len(dataset)))
    print("Variation types: ", count_variations)
    assert np.sum(count_variations) == len(dataset) / 2
    return dataset

# Example usage:
num_samples = 1000
max_length = 20
dataset = create_dataset(num_samples, max_length)
print(dataset[:5])


Dataset created with 2000 samples.
Variation types:  [733, 147, 120]
[('ac', 0), ('aaaaaabbbbbbcccccc', 1), ('bab', 0), ('bacb', 0), ('aabbcc', 1)]


## Preprocessing

In [440]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


# define char to index mapping
char_to_index = {'a': 0, 'b': 1, 'c': 2, 'p': 3}

def char_to_one_hot(char, num_classes=3):
    index = char_to_index[char]
    one_hot = np.zeros(num_classes)
    one_hot[index] = 1
    return one_hot

def string_to_tensor(string, padding_length=0, add_padding = False):
    if add_padding:
        tensor = np.array([char_to_one_hot(char, num_classes=4) for char in string]
                          + [char_to_one_hot('p', num_classes=4)] * (padding_length - len(string)))
    else:
        tensor = np.array([char_to_one_hot(char) for char in string])
    return torch.tensor(tensor, dtype=torch.float32)

class GrammarDataset(Dataset):
    def __init__(self, data, padding_length=0, add_padding = False):
        self.data = data
        self.add_padding = add_padding
        self.padding_length = padding_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        string, label = self.data[idx]
        input_tensor = string_to_tensor(string, padding_length=self.padding_length, add_padding=self.add_padding)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        return input_tensor, label_tensor

# Example usage
dataset = create_dataset(10000, max_length=20)
add_padding = True
if add_padding:
    grammar_dataset = GrammarDataset(dataset, padding_length=20, add_padding=True)
else:
    grammar_dataset = GrammarDataset(dataset)

#print example
print(grammar_dataset[0])



Dataset created with 20000 samples.
Variation types:  [7031, 1602, 1367]
(tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]]), tensor(1.))


In [441]:
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(0)    
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)  # Initial hidden state
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Get the output from the last time step
        out = self.sigmoid(out)
        return out


In [442]:
import torch.optim as optim

# Training loop method
def train_model(model, criterion, optimizer, num_epochs, dataset, batch_size=1):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        for inputs, labels in data_loader:
            inputs, labels = inputs.squeeze(0), labels.unsqueeze(0)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.squeeze())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(data_loader)}')

In [443]:
# Hyperparameters
hidden_size = 2
output_size = 1
num_epochs = 10
learning_rate = 0.001


In [444]:
def evaluate(model, dataset, batch_size=1):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in DataLoader(dataset, batch_size):
            inputs, labels = inputs.squeeze(0), labels.unsqueeze(0)
            outputs = model(inputs)

            outputs = outputs.squeeze()
            labels = labels.squeeze()
            assert labels.size() == outputs.size()
            
            predicted = (outputs > 0.5).float()
            
            if labels.size() == torch.Size([]):
                total += 1
                correct += (predicted == labels).item()
            else:
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy


In [445]:
## Create the dataset and use it for all models
dataset = create_dataset(1000, max_length=20)

# With padding
grammar_dataset_padded = GrammarDataset(dataset, padding_length=20, add_padding=True)

# Without padding
grammar_dataset = GrammarDataset(dataset)

# Train and test splits
train_size = int(0.8 * len(grammar_dataset))
test_size = len(grammar_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(grammar_dataset, [train_size, test_size])

train_size_padded = int(0.8 * len(grammar_dataset_padded))
test_size_padded = len(grammar_dataset_padded) - train_size_padded
train_dataset_padded, test_dataset_padded = torch.utils.data.random_split(grammar_dataset_padded, [train_size_padded, test_size_padded])


Dataset created with 2000 samples.
Variation types:  [721, 155, 124]


In [446]:
# Run without padding
add_padding = False
batch_size = 1
input_size = 3

# model definition
model = RNNClassifier(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train_model(model, criterion, optimizer, num_epochs, train_dataset, batch_size)    
accuracy = evaluate(model, test_dataset, batch_size)
print(f'Accuracy: {accuracy * 100:.2f}%')
model.eval()
with torch.no_grad():
    for i in range(10):
        inputs, labels = test_dataset[i]
        inputs = inputs.unsqueeze(0)
        outputs = model(inputs)
        predicted = outputs
        inputs_str = ''.join(['a' if c[0] == 1 else 'b' if c[1] == 1 else 'c' if c[2] == 1 else '' for c in inputs[0]])
        print(f'Input: {inputs_str}, Label: {int(labels.item())}, Predicted: {predicted.item()}')
        
        
# Test 'abbc' as input
inputs = string_to_tensor('abbc')
inputs = inputs.unsqueeze(0)
outputs = model(inputs)
predicted = outputs
inputs_str = ''.join(['a' if c[0] == 1 else 'b' if c[1] == 1 else 'c' if c[2] == 1 else '' for c in inputs[0]])
print(f'Input: {inputs_str}, Predicted: {predicted.item()}')


Epoch [1/10], Loss: 0.6385653500258922
Epoch [2/10], Loss: 0.5237812813464552
Epoch [3/10], Loss: 0.48006012875121085
Epoch [4/10], Loss: 0.46463105104165153
Epoch [5/10], Loss: 0.45482728651259097
Epoch [6/10], Loss: 0.4469174789043609
Epoch [7/10], Loss: 0.43487689760164355
Epoch [8/10], Loss: 0.4238006698561367
Epoch [9/10], Loss: 0.41492007648863366
Epoch [10/10], Loss: 0.40758682242012584
Accuracy: 78.75%
Input: aaabbbccc, Label: 1, Predicted: 0.773736834526062
Input: aabbcc, Label: 1, Predicted: 0.7239737510681152
Input: cacbb, Label: 0, Predicted: 0.012911645695567131
Input: aabbcc, Label: 1, Predicted: 0.7239737510681152
Input: ba, Label: 0, Predicted: 0.0513872355222702
Input: aaaaaaabbbbbbccccccc, Label: 0, Predicted: 0.7920834422111511
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.7918806672096252
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.7918806672096252
Input: aaabbbccc, Label: 1, Predicted: 0.773736834526062
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.79

In [447]:
# With Padding
add_padding = True
batch_size = 32
input_size = 4

model = RNNClassifier(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_model(model, criterion, optimizer, num_epochs, train_dataset_padded, batch_size)
accuracy = evaluate(model, test_dataset_padded, batch_size)
print(f'Accuracy: {accuracy * 100:.2f}%')
model.eval()

with torch.no_grad():
    for i in range(10):
        inputs, labels = test_dataset_padded[i]
        inputs = inputs.unsqueeze(0)
        outputs = model(inputs)
        predicted = outputs
        inputs_str = ''.join(['a' if c[0] == 1 else 'b' if c[1] == 1 else 'c' if c[2] == 1 else '' for c in inputs[0]])
        print(f'Input: {inputs_str}, Label: {int(labels.item())}, Predicted: {predicted.item()}')

Epoch [1/10], Loss: 0.697312206029892
Epoch [2/10], Loss: 0.6938045907020569
Epoch [3/10], Loss: 0.6923642420768737
Epoch [4/10], Loss: 0.6920537102222443
Epoch [5/10], Loss: 0.6913264262676239
Epoch [6/10], Loss: 0.6906245183944703
Epoch [7/10], Loss: 0.6899545621871949
Epoch [8/10], Loss: 0.688954520225525
Epoch [9/10], Loss: 0.6876797449588775
Epoch [10/10], Loss: 0.6860293436050415
Accuracy: 58.50%
Input: aaaaabbbbbbcccccc, Label: 0, Predicted: 0.4934021830558777
Input: cabbb, Label: 0, Predicted: 0.4928710460662842
Input: bbca, Label: 0, Predicted: 0.4928710460662842
Input: b, Label: 0, Predicted: 0.4928710460662842
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.5295969843864441
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.5295969843864441
Input: aaaaaa, Label: 0, Predicted: 0.4928710460662842
Input: aabbc, Label: 0, Predicted: 0.49287116527557373
Input: aaaaabbbbbccccc, Label: 1, Predicted: 0.49431759119033813
Input: aaaabbbbcccc, Label: 1, Predicted: 0.4929177165031433


## LSTM Model

In [448]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(0)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Take the last time step's output
        out = self.fc(out)
        return torch.sigmoid(out)


In [449]:
# Without Padding
add_padding = False
batch_size = 1
input_size = 3

model = LSTMClassifier(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_model(model, criterion, optimizer, num_epochs, train_dataset, batch_size)
accuracy = evaluate(model, test_dataset, batch_size)
print(f'Accuracy: {accuracy * 100:.2f}%')
model.eval()

with torch.no_grad():
    for i in range(10):
        inputs, labels = test_dataset[i]
        inputs = inputs.unsqueeze(0)
        outputs = model(inputs)
        predicted = outputs
        inputs_str = ''.join(['a' if c[0] == 1 else 'b' if c[1] == 1 else 'c' if c[2] == 1 else '' for c in inputs[0]])
        print(f'Input: {inputs_str}, Label: {int(labels.item())}, Predicted: {predicted.item()}')
        

Epoch [1/10], Loss: 0.6552417316660285
Epoch [2/10], Loss: 0.5373924931883812
Epoch [3/10], Loss: 0.4861632794234902
Epoch [4/10], Loss: 0.4585159652819857
Epoch [5/10], Loss: 0.44248267602175473
Epoch [6/10], Loss: 0.4260656095319428
Epoch [7/10], Loss: 0.40878328911261635
Epoch [8/10], Loss: 0.4100137776811607
Epoch [9/10], Loss: 0.38430937006138266
Epoch [10/10], Loss: 0.3727330004714895
Accuracy: 81.75%
Input: aaabbbccc, Label: 1, Predicted: 0.7811494469642639
Input: aabbcc, Label: 1, Predicted: 0.6414744257926941
Input: cacbb, Label: 0, Predicted: 0.04388630390167236
Input: aabbcc, Label: 1, Predicted: 0.6414744257926941
Input: ba, Label: 0, Predicted: 0.06045932322740555
Input: aaaaaaabbbbbbccccccc, Label: 0, Predicted: 0.8243005871772766
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.8242754936218262
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.8242754936218262
Input: aaabbbccc, Label: 1, Predicted: 0.7811494469642639
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.824

In [450]:
# With Padding
add_padding = True
batch_size = 32
input_size = 4

model = LSTMClassifier(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_model(model, criterion, optimizer, num_epochs, train_dataset_padded, batch_size)
accuracy = evaluate(model, test_dataset_padded, batch_size)
print(f'Accuracy: {accuracy * 100:.2f}%')
model.eval()

with torch.no_grad():
    for i in range(5):
        inputs, labels = test_dataset_padded[i]
        inputs = inputs.unsqueeze(0)
        outputs = model(inputs)
        predicted = outputs
        inputs_str = ''.join(['a' if c[0] == 1 else 'b' if c[1] == 1 else 'c' if c[2] == 1 else '' for c in inputs[0]])
        print(f'Input: {inputs_str}, Label: {int(labels.item())}, Predicted: {predicted.item()}')
        
        

Epoch [1/10], Loss: 0.7655710363388062
Epoch [2/10], Loss: 0.7454549372196198
Epoch [3/10], Loss: 0.7273312842845917
Epoch [4/10], Loss: 0.712902443408966
Epoch [5/10], Loss: 0.7040875494480133
Epoch [6/10], Loss: 0.6991366815567016
Epoch [7/10], Loss: 0.6963172829151154
Epoch [8/10], Loss: 0.6948109173774719
Epoch [9/10], Loss: 0.6939846813678742
Epoch [10/10], Loss: 0.6935606038570404
Accuracy: 46.25%
Input: aaaaabbbbbbcccccc, Label: 0, Predicted: 0.48046326637268066
Input: cabbb, Label: 0, Predicted: 0.4820120930671692
Input: bbca, Label: 0, Predicted: 0.48200806975364685
Input: b, Label: 0, Predicted: 0.4820096790790558
Input: aaaaaabbbbbbcccccc, Label: 1, Predicted: 0.4789251685142517


In [451]:
# test both models with a longer example
string = 'abcc'
inputs = string_to_tensor(string, padding_length=20, add_padding=True).unsqueeze(0)
outputs = model(inputs)
predicted = outputs
print(f'Input: {string}, Predicted: {predicted.item()}')




Input: abcc, Predicted: 0.4820011258125305
